# Agentic AI

## Prerequisites

### Install a Local LLM with Ollama

To run this project locally, we will install and use **Ollama**, a lightweight runtime for local large language models.

**Download Ollama:**  
https://ollama.com/

Once installed, you can pull any model you want to run.  
Below are a few recommended examples, but you are free to pick any size or model from the Ollama library.

ollama pull qwen3:0.6b

or

ollama pull ibm/granite4:350m

or

Choose any model you prefer, make sure the model supports tools.
Browse available models here:
https://ollama.com/library



### Python requirements

In [240]:
#!pip install langgraph langchain-google-genai langchain-core mcp langchain-ollama

## 1. Define FastMCP Tools

In [241]:
from heuristics import *
from color_blocks_state import *
from search import *
import asyncio
from langchain_core.tools import StructuredTool

In [242]:
import re
from typing import Any

def normalize_start_blocks_any(x: Any) -> str:
    """
    Returns canonical "(a,b),(c,d),..." string.
    Accepts:
      - str: "(5,2),(1,3)..."
      - list[str] chunks
      - list[tuple[int,int]] or tuple[tuple[int,int],...]
    """
    # tuple/list of pairs -> convert directly
    if isinstance(x, (list, tuple)) and x and all(isinstance(t, (list, tuple)) and len(t) == 2 for t in x):
        pairs = [f"({int(a)},{int(b)})" for a, b in x]
        return ",".join(pairs)

    # list of strings -> join
    if isinstance(x, list):
        s = ",".join(str(p) for p in x)
    else:
        s = str(x)

    s = s.strip().lstrip("/").strip().strip('"').strip("'")

    # extract (d,d) pairs
    pairs = re.findall(r"\(\s*\d+\s*,\s*\d+\s*\)", s)
    if not pairs:
        raise ValueError(f"start_blocks invalid: expected '(a,b),(c,d),...' got {x!r}")
    return ",".join(p.replace(" ", "") for p in pairs)


def normalize_goal_blocks_any(x: Any) -> str:
    """
    Normalize goal_blocks into canonical "a,b,c,..." string.

    Accepts artifacts like:
      - "/10,2,3,4"
      - "/2,/4,/6,/8"   <-- per-token slashes
      - quotes, spaces
      - list/tuple of ints/strings
    """
    # list/tuple of ints or digit-strings -> join
    if isinstance(x, (list, tuple)) and x and all(
        isinstance(v, int) or (isinstance(v, str) and v.strip().lstrip("/").isdigit())
        for v in x
    ):
        return ",".join(str(int(str(v).strip().lstrip("/"))) for v in x)

    s = str(x).strip()

    # strip wrappers/quotes
    s = s.strip('"').strip("'")
    s = s.replace(" ", "")

    # If it looks like weird nested tuples, reject clearly
    if s.startswith("[(") or s.startswith("(("):
        raise ValueError(f"goal_blocks invalid: expected 'a,b,c,...' got {x!r}")

    # split by comma, strip leading '/' from EACH token
    tokens = [t for t in s.split(",") if t != ""]
    cleaned = []
    for t in tokens:
        t2 = t.strip().lstrip("/")   # <-- key fix
        if not t2.isdigit():
            raise ValueError(f"goal_blocks invalid: expected 'a,b,c,...' got {x!r}")
        cleaned.append(t2)

    if not cleaned:
        raise ValueError(f"goal_blocks invalid: expected 'a,b,c,...' got {x!r}")

    return ",".join(cleaned)


In [243]:
from mcp.server.fastmcp import FastMCP
import math

# Initialize FastMCP
mcp = FastMCP("Unified Solver")

@mcp.tool()
def calculate_sum(a: float, b: float) -> float:
    """Calculates the sum of two numbers."""
    return a + b

@mcp.tool()
def calculate_power(base: float, exponent: float) -> float:
    """Calculates the power of a base number."""
    return math.pow(base, exponent)

# TO DO: Add more tools as needed for your application


@mcp.tool()
def solve_cost_mcp(start_blocks, goal_blocks) -> float:
    """
    Compute the optimal solution cost for the Color Blocks search problem.

    INPUTS (MUST be single values, not nested objects):
    - start_blocks: string "(a,b),(c,d),..."  (each block is a pair top,bottom)
    - goal_blocks:  string "g0,g1,g2,..." (top colors required at each position)

    OUTPUT:
    - A single number (float): optimal path cost (each move costs 1).
    """
    start_blocks = normalize_start_blocks_any(start_blocks)
    goal_blocks  = normalize_goal_blocks_any(goal_blocks)

    init_goal_for_heuristics(goal_blocks)
    init_goal_for_search(goal_blocks)
    start_state = color_blocks_state(blocks_str=start_blocks)
    path = search(start_state, advanced_heuristic)
    return float("inf") if path is None else float(path[-1].g)

def solve_cost_wrapper(start_blocks: str, goal_blocks: str) -> float:
    # Just delegate to your MCP tool function
    return solve_cost_mcp(start_blocks, goal_blocks)


def as_langchain_tool_from_fn(fn, name: str, description: str):
    return StructuredTool.from_function(
        func=fn,
        name=name,
        description=description,
        coroutine=fn if asyncio.iscoroutinefunction(fn) else None,
    )


solve_cost_tool = as_langchain_tool_from_fn(
    solve_cost_wrapper,
    name="solve_color_blocks_cost",
    description="""
    Compute the optimal solution cost for the Color Blocks search problem.

    INPUTS (MUST be single values, not nested objects):
    - start_blocks: string "(a,b),(c,d),..."  (each block is a pair top,bottom)
    - goal_blocks:  string "g0,g1,g2,..." (top colors required at each position)

    OUTPUT:
    - A single number (float): optimal path cost (each move costs 1).
    """
)



## 2. LLM + MCP

### 2.1. Global instance of our LLM

In [244]:
import os
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI


# Choose your model here, can be Ollama or Google Gemini
# Can also switch between different model sizes as needed
model = "qwen3:0.6b"
model = "ibm/granite4:350m"
global_llm = ChatOllama(model=model, temperature=0.0)


# SETUP API KEY if using Google Gemini
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY_HERE"

# model = "gemini-2.5-flash"
# model = "gemini-2.5-flash-lite"
# global_llm = ChatGoogleGenerativeAI(model=model, temperature=0)


### 2.2. Our agent graph

In [245]:
from langgraph.graph import MessagesState, START, StateGraph
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver # Optional: For saving graph state


def create_agent_graph(sys_msg, tools):
    """ Creates a LangGraph StateGraph with the given tools integrated."""

    llm = global_llm

    if tools:
        llm_with_tools = llm.bind_tools(tools)
    else:
        llm_with_tools = llm

    # Node
    def assistant(state: MessagesState):
        return {
            "messages": [
                llm_with_tools.invoke([sys_msg] + state["messages"], think=False)
            ]
        }

    # Graph
    builder = StateGraph(MessagesState)

    # Define the basic graph structure
    builder.add_node("assistant", assistant)
    builder.add_edge(START, "assistant")

    if tools:
        builder.add_node("tools", ToolNode(tools))  
        builder.add_conditional_edges(
            "assistant",
            tools_condition,
        )
        builder.add_edge("tools", "assistant")

    react_graph = builder.compile()

    return react_graph


async def run_agent(prompt, tools, sys_msg=""):

    sys_msg = SystemMessage(content=sys_msg)

    # 3. Create Graph
    graph = create_agent_graph(sys_msg, tools)
    
    # 4. Run (using ainvoke for async tools)
    config = {"configurable": {"thread_id": "1"}}
    result = await graph.ainvoke({"messages": [HumanMessage(content=prompt)]}, config)

    last_msg = result["messages"][-1].content

    # Extract tool names and outputs
    tools_used = []
    tools_output = []
    
    # Parsing logic specific to your request
    for msg in result["messages"]:
        # In LangChain, tool calls are usually in 'tool_calls' attribute of AIMessage
        # or 'name' attribute if it is a ToolMessage
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
             for tool_call in msg.tool_calls:
                tools_used.append(tool_call['name'])
        
        if msg.type == 'tool':
            tools_output.append(msg.content)

    return last_msg, tools_used, tools_output

### 2.3. Tools that run spacific agent (with tools and without)

In [246]:

from langchain_core.tools import StructuredTool
import asyncio

WITH_TOOLS_SYS = """
You are the WITH-TOOLS assistant.

You MUST call the tool solve_cost_mcp EXACTLY ONCE using the given arguments.

Input format (use EXACTLY as provided):
- start_blocks: a single string in the format "(a,b),(c,d),..."
- goal_blocks: a single string in the format "x,y,z,..."

Rules:
- Do NOT add or remove characters.
- Do NOT add leading symbols (/, [, ], quotes).
- Do NOT split start_blocks into a list.
- Do NOT change the order of blocks.

After the tool returns:
- Output ONLY the numeric solution cost.
- Do NOT include explanations or extra text.
"""

NO_TOOLS_SYS = """
You are the NO-TOOLS assistant.

You are solving a SEARCH problem WITHOUT tools and WITHOUT running code.
You must still produce an answer.

Domain rules:
- State is a list of pairs (top,bottom).
- spin(i): swaps top and bottom of block i. Cost = 1.
- flip(i): reverses the order of blocks from index i to the end. Cost = 1.
- Goal: for each position i, the top color equals goal_blocks[i].

Instructions:
- Reason mentally using the rules above.
- If unsure, prefer a CONSERVATIVE estimate (slightly higher, not lower).
- Never refuse and never ask for more information.

Output:
- Output ONLY a single number representing the estimated solution cost.
"""


@mcp.tool()
async def ask_agent_with_tools(start_blocks: str, goal_blocks: str) -> str:
    """
    WITH-TOOLS assistant.
    Must call solve_cost_mcp exactly once and return ONLY the numeric cost.
    """

    sys_msg = f"""{WITH_TOOLS_SYS}

Use these exact inputs:
start_blocks = "{start_blocks}"
goal_blocks  = "{goal_blocks}"
"""

    # Only the solver tool is available
    tools = [solve_cost_mcp]

    last_msg, _, _ = await run_agent(
        prompt="Compute the solution cost.",
        tools=tools,
        sys_msg=sys_msg
    )

    return last_msg

@mcp.tool()
async def ask_agent_without_tools(start_blocks: str, goal_blocks: str) -> str:
    """
    NO-TOOLS assistant.
    Must estimate and return ONLY a numeric cost.
    """

    sys_msg = f"""{NO_TOOLS_SYS}

Start blocks: {start_blocks}
Goal blocks: {goal_blocks}
"""

    last_msg, _, _ = await run_agent(
        prompt="Estimate the minimal solution cost. Output only a number.",
        tools=[],
        sys_msg=sys_msg
    )

    return last_msg


def as_langchain_tool(mcp_fn, *, name: str, description: str):
    """
    Wrap an @mcp.tool() function (sync or async) as a LangChain tool
    so it can be used inside LangGraph ToolNode.
    """
    return StructuredTool.from_function(
        func=mcp_fn,
        name=name,
        description=description,
        coroutine=mcp_fn if asyncio.iscoroutinefunction(mcp_fn) else None,
    )


async def ask_with_tools_wrapper(start_blocks: str, goal_blocks: str) -> str:
    return await ask_agent_with_tools(start_blocks, goal_blocks)

async def ask_without_tools_wrapper(start_blocks: str, goal_blocks: str) -> str:
    return await ask_agent_without_tools(start_blocks, goal_blocks)

ask_with_tools_tool = as_langchain_tool_from_fn(
    ask_with_tools_wrapper,
    name="ask_agent_with_tools",
    description="Calls the with-tools assistant and returns its numeric cost."
)

ask_without_tools_tool = as_langchain_tool_from_fn(
    ask_without_tools_wrapper,
    name="ask_agent_without_tools",
    description="Calls the no-tools assistant and returns its numeric cost."
)




## 3. Run the Test

In [247]:
sys_msg = """
You are a judge comparing two assistants on the SAME Color Blocks instance.

You MUST do these steps in order:
1) Call ask_agent_with_tools(start_blocks, goal_blocks) exactly once.
2) Call ask_agent_without_tools(start_blocks, goal_blocks) exactly once.
3) Read both results and extract one numeric cost from each (if missing -> NA).
4) Decide who is better using ONLY the decision rule below.

Decision rule ("Better"):
- If tool-based returns a valid finite number -> Better = tool-based.
- No-tools is Better ONLY if tool-based fails (error/empty/NA/inf/NaN/non-numeric) AND no-tools returns a valid finite number.
- Lower cost does NOT automatically mean better, because no-tools may under-estimate.
- If tool-based cost is lower than the no-tools cost, tool-based is definitely better.

STRICT OUTPUT RULES:
- Output EXACTLY 4 lines (no extra lines).
- In the Better line, output EXACTLY ONE of these two literals: tool-based OR no-tools (no other text, no separators, no "|").
- The Reason line MUST be non-empty and MUST mention why the better assistant won (e.g., "tool-based used the solver" or "tool-based failed and no-tools returned a number").
- Do NOT leave Reason blank. Do NOT output placeholders.

Output EXACTLY 4 lines:
Tool-based cost: <number-or-NA>
No-tools cost: <number-or-NA>
Better: <tool-based or no-tools>
Reason: <one short sentence with a concrete justification>
"""



# start_blocks = "(5,2),(1,3),(9,22),(21,4)"
# goal_blocks  = "2,22,4,3"

# judge_prompt = f"""
# Call the two tools using these EXACT arguments:
# start_blocks = "{start_blocks}"
# goal_blocks  = "{goal_blocks}"

# Do not change formatting. Do not split strings into lists.
# """

# tool_list = [ask_with_tools_tool, ask_without_tools_tool]
# response, tools_used, outputs = await run_agent(judge_prompt, tool_list, sys_msg)

# print("=== RESPONSE ===")
# print(response)

# print("=== TOOLS USED ===")
# print(tools_used)

# print("=== TOOL OUTPUTS ===")
# print(outputs)

test_cases = [
    ("(5,2),(1,3),(9,22),(21,4)", "2,22,4,3"),
    ("(1,2),(3,4),(5,6),(7,8)", "1,3,5,7"),
    ("(2,1),(4,3),(6,5),(8,7)", "2,4,6,8"),
    ("(10,1),(2,20),(3,30),(4,40)", "10,2,3,4"),
    ("(8,9),(7,6),(5,4),(3,2)", "8,7,5,3"),
]

for i, (start_blocks, goal_blocks) in enumerate(test_cases, 1):
    print(f"\n================= CASE {i} =================")
    judge_prompt = f"""
    Call BOTH tools using these EXACT arguments (single strings):
    start_blocks = "{start_blocks}"
    goal_blocks  = "{goal_blocks}"

    After you get BOTH tool results, output the 4 required lines including a non-empty Reason.
    """

    tool_list = [ask_with_tools_tool, ask_without_tools_tool]
    response, tools_used, outputs = await run_agent(judge_prompt, tool_list, sys_msg)

    print("=== RESPONSE ===")
    print(response)
    print("=== TOOLS USED ===")
    print(tools_used)
    print("=== TOOL OUTPUTS ===")
    print(outputs)



================= CASE 1 =================


HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== RESPONSE ===
Tool-based used the solver with a cost of 6. Reason: tool-based was lower than no-tools cost.
=== TOOLS USED ===
['ask_agent_with_tools', 'ask_agent_without_tools']
=== TOOL OUTPUTS ===
["Error invoking tool 'ask_agent_with_tools' with kwargs {'start_blocks': '(5,2),(1,3),(9,22),(21,4)'} with error:\n goal_blocks: Field required\n Please fix the error and try again.", 'The minimal solution cost is 6.']

================= CASE 2 =================


HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== RESPONSE ===
Tool-based cost: 4  
No-tools cost: 0  
Better: tool-based
=== TOOLS USED ===
['ask_agent_with_tools', 'ask_agent_without_tools']
=== TOOL OUTPUTS ===
["Error invoking tool 'ask_agent_with_tools' with kwargs {'start_blocks': '(1,2),(3,4),(5,6),(7,8)'} with error:\n goal_blocks: Field required\n Please fix the error and try again.", '1. The top color must be 1 to match goal_blocks[0].  \n2. Flip block\u202f1 (cost\u202f1).  \n3. The remaining blocks can be flipped in order: block\u202f5 → block\u202f6, block\u202f7 → block\u202f8. Each flip costs 1.  \n4. Total cost = 1 + 1 + 1 + 1 = **4**.']

================= CASE 3 =================


...

=== RESPONSE ===
Tool-based assistant used the solver to find a solution with a cost of 0. The no-tools assistant failed and returned an error/empty/NA/inf/NaN/non-numeric, but tool-based did not fail. Therefore, tool-based was better.
=== TOOLS USED ===
['ask_agent_with_tools', 'ask_agent_without_tools']
=== TOOL OUTPUTS ===
['The optimal solution cost is 0.0.', 'The minimal solution cost is 0.']

================= CASE 4 =================


HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== RESPONSE ===
Tool-based assistant used the solver to find a solution with a cost of 0. The no-tools assistant failed and returned an error/empty/NA/inf/NaN/non-numeric, but tool-based did not fail. Therefore, tool-based was better.
=== TOOLS USED ===
['ask_agent_with_tools', 'ask_agent_without_tools']
=== TOOL OUTPUTS ===
['The optimal solution cost is 0.0.', 'The minimal solution cost is 0.']

================= CASE 5 =================


HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


=== RESPONSE ===
Tool-based cost: 2  
No-tools cost: 0  
Better: tool-based
=== TOOLS USED ===
['ask_agent_with_tools', 'ask_agent_without_tools']
=== TOOL OUTPUTS ===
['The optimal solution cost is 0.0.', '1. First block is already goal, so no flips needed.\n2. Second block can be flipped to match goal (cost = 0).\n3. Third block needs to swap with the first block: total cost = 1 + 1 = **2**.\n4. Fourth block matches goal, so no flips or swaps required.\n\nEstimated solution cost: **2**.']
